# Bitcoin and XRT token analysis

In [57]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
import hvplot.pandas
import plotly.express as px
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

%matplotlib inline

## Pull Crypto prices

In [58]:
# Get prices of Bitcoin and XRT in usd
cg.get_price(ids=['bitcoin', 'ripple'], vs_currencies='usd')

{'bitcoin': {'usd': 63007}, 'ripple': {'usd': 1.12}}

In [59]:
# Define function to pull cryptocurrency prices from Coingecko

def crypto_pull(ticker, name):
    # Pull USD prices for cryptocurrency
    ticker = cg.get_coin_market_chart_by_id(id=name, vs_currency="usd", days="max")
    
    # Isolate target data (prices)
    data = ticker["prices"]
    
    # Create new dataframe and assign column names
    df = pd.DataFrame(data, columns=["Date", "Close"])
    
    # Convert date format from UTC to YYYY-MM-DD format
    df['Date'] = pd.to_datetime(df["Date"], unit="ms").apply(lambda x: x.date())
    
    # Round closing prices to two decimals
    df["Close"] = df["Close"].round(2)
    
    # Set "Date" as index
    df.set_index("Date", inplace = True)
    
    return df

In [60]:
# Get Bitcoin prices
btc = crypto_pull('btc', 'bitcoin')
btc.rename(columns={'Close':'btc'}, inplace=True)
btc.head()

,btc
Date,
2013-04-28,135.30
2013-04-29,141.96
2013-04-30,135.30
2013-05-01,117.00
2013-05-02,103.43


In [61]:
# Get ripple prices
xrp = crypto_pull('xrp', 'ripple')
xrp.rename(columns={'Close':'xrp'}, inplace=True)
xrp.head()

,xrp
Date,
2013-08-04,0.01
2013-08-05,0.01
2013-08-06,0.00
2013-08-07,0.00
2013-08-08,0.00


In [62]:
# Concat btc and xrp DataFrames
all_returns = pd.concat([btc, xrp], axis='columns', join='inner')
all_returns.head()

,btc,xrp
Date,,
2013-08-04,104.86,0.01
2013-08-05,106.56,0.01
2013-08-06,105.84,0.00
2013-08-07,106.75,0.00
2013-08-08,101.67,0.00


In [63]:
# Create dataframe of closing prices for BTC and XRP with ranges between '2017-10-26' and '2021-10-26'
startdate = pd.to_datetime('2017-10-26').date()
enddate = pd.to_datetime('2021-10-26').date()

new_df = all_returns.loc[startdate:enddate]
new_df.head()

,btc,xrp
Date,,
2017-10-26,5883.05,0.2
2017-10-27,5775.52,0.2
2017-10-28,5734.31,0.2
2017-10-29,6123.55,0.2
2017-10-30,6104.39,0.2


In [64]:
# Calculate daily returns for each portfolio
daily_returns = new_df.pct_change().dropna()
daily_returns.head()

,btc,xrp
Date,,
2017-10-27,-0.018278,0.0
2017-10-28,-0.007135,0.0
2017-10-29,0.067879,0.0
2017-10-30,-0.003129,0.0
2017-10-31,0.043290,0.0


In [65]:
# Calculate cumulative returns for each portfolio
cumulative_returns = (1 + daily_returns).cumprod()
cumulative_returns.head()

,btc,xrp
Date,,
2017-10-27,0.981722,1.0
2017-10-28,0.974717,1.0
2017-10-29,1.040880,1.0
2017-10-30,1.037623,1.0
2017-10-31,1.082542,1.0


## Evaluate BTC and XRP performance

In [66]:
# Create Evaluation dataframe
metrics = ['Annual Return', 'Annual Volatility', 'Sharpe Ratio','Sortino Ratio']
columns = ['btc_values', 'xrp_values']

# Initialize the DataFrame with index set to evaluation metrics and columns as 'btc_values', 'xrp_values'
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,btc_values,xrp_values
Annual Return,NaN,NaN
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [67]:
# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return', 'btc_values'] = (daily_returns['btc'].mean() * 252)
portfolio_evaluation_df.loc['Annual Return', 'xrp_values'] = (daily_returns['xrp'].mean() * 252)
portfolio_evaluation_df

,btc_values,xrp_values
Annual Return,0.631725,0.914547
Annual Volatility,NaN,NaN
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [68]:
# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility', 'btc_values'] = (daily_returns['btc'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Annual Volatility', 'xrp_values'] = (daily_returns['xrp'].std() * np.sqrt(252))
portfolio_evaluation_df

,btc_values,xrp_values
Annual Return,0.631725,0.914547
Annual Volatility,0.663997,1.158434
Sharpe Ratio,NaN,NaN
Sortino Ratio,NaN,NaN


In [69]:
# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio', 'btc_values'] = (daily_returns['btc'].mean() * 252) / (daily_returns['btc'].std() * np.sqrt(252))
portfolio_evaluation_df.loc['Sharpe Ratio', 'xrp_values'] = (daily_returns['xrp'].mean() * 252) / (daily_returns['xrp'].std() * np.sqrt(252))
portfolio_evaluation_df

,btc_values,xrp_values
Annual Return,0.631725,0.914547
Annual Volatility,0.663997,1.158434
Sharpe Ratio,0.951397,0.789469
Sortino Ratio,NaN,NaN


In [70]:
# Calculate Downside Return
sortino_ratio_df = daily_returns.copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask_btc = sortino_ratio_df['btc'] < target
mask_xrp = sortino_ratio_df['xrp'] < target
sortino_ratio_df.loc[mask_btc, 'Downside Returns'] = sortino_ratio_df['btc']**2
sortino_ratio_df.loc[mask_xrp, 'Downside Returns'] = sortino_ratio_df['xrp']**2

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return_btc = sortino_ratio_df['btc'].mean() * 252
expected_return_xrp = sortino_ratio_df['xrp'].mean() * 252
sortino_ratio_btc = expected_return_btc/down_stdev
sortino_ratio_xrp = expected_return_xrp/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio', 'btc_values'] = sortino_ratio_btc
portfolio_evaluation_df.loc['Sortino Ratio', 'xrp_values'] = sortino_ratio_xrp
portfolio_evaluation_df

,btc_values,xrp_values
Annual Return,0.631725,0.914547
Annual Volatility,0.663997,1.158434
Sharpe Ratio,0.951397,0.789469
Sortino Ratio,0.947337,1.371458
